C. elegans Lineage Data
- Title:    A lineage-resolved molecular atlas of C. elegans embryogenesis at single-cell resolution
- Author:   Packer et al.
- Year:     2019
- doi:      https://doi.org/10.1126/science.aax1971
- GEO:      https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE126954
- file:
  - `.csv`: Supplementary file -> `cell_annotation.csv.gz` -> Column: `lineage`;
  - ~~`.h5ad`: https://zenodo.org/records/15236812~~

Worm Atlas Complete Cell List
- url:      https://www.wormatlas.org/celllistsulston.html
- ref:      Sulston, JE and White, JG (1988), "Parts list", in "The Nematode Caenorhabditis elegans, eds WB Wood et al, Cold Spring Harbor Laboratory Press, Cold Spring Harbor, New York, USA, pp 415 - 431.
  - Title:  The embryonic cell lineage of the nematode Caenorhabditis elegans
  - Author: J.E. Sulston, E. Schierenberg, J.G. White, J.N. Thomson
  - Year:   1983
  - doi:    https://doi.org/10.1016/0012-1606(83)90201-4

Method
1. Generate Tree.
2. a

In [26]:
import os
os.chdir('/Users/yongjunchoi/Documents/GitHub/Research/Research')

# Packer et al., (2019)

## 0. Read `.gz` -> `.nwk` Tree

In [32]:
import pandas as pd
import gzip

file_path = "Polytomy/Codes/Data/GSE126954_cell_annotation.csv.gz"
output_path = "Polytomy/Codes/Data/lineages.csv"

# 1. 데이터 읽기
df = pd.read_csv(file_path, compression="gzip")

# 2. lineage 컬럼 가져오기, 결측 제외
lineages = df['lineage'].dropna()

processed = []

for lineage in lineages:
    # ':' 뒤 제거
    lineage = lineage.split(':')[0]
    # '/' 기준으로 분리
    parts = lineage.split('/')
    processed.extend(parts)

processed_unique_sorted = sorted(set(processed))

# 3. 결과를 DataFrame으로 만들어 csv로 저장
processed_df = pd.DataFrame(processed_unique_sorted, columns=['lineage'])
processed_df.to_csv(output_path, index=False)

print(f"저장 완료: {output_path}")

저장 완료: Polytomy/Codes/Data/lineages.csv


In [ ]:
import pandas as pd
import gzip

file_path = "Polytomy/Codes/Data/GSE126954_cell_annotation.csv.gz"

# 0. pandas에서 gzip 바로 읽기
cell_anno = pd.read_csv(file_path, compression='gzip')

# 1. 데이터 읽기
df = pd.read_csv(file_path, compression="gzip")
lineages = df['lineage'].dropna().unique()  # 결측 제외, 고유 lineage

# 2. lineage -> Newick tree 변환 (대문자 덩어리, 소문자 개별 노드)
def lineage_to_newick(lineages):
    root = Tree(name="zygote")
    
    for lin in lineages:
        # '/'가 있으면 여러 경로로 분리
        paths = lin.split('/')
        for p in paths:
            node = root
            i = 0
            while i < len(p):
                # 대문자인 경우: 연속 대문자 덩어리 추출
                if p[i].isupper():
                    j = i + 1
                    while j < len(p) and p[j].isupper():
                        j += 1
                    chunk = p[i:j]  # 대문자 덩어리
                    i = j
                    # 자식 존재 여부 확인 후 생성
                    existing_child = next((c for c in node.children if c.name == chunk), None)
                    if existing_child:
                        node = existing_child
                    else:
                        node = node.add_child(name=chunk)
                else:
                    # 소문자는 글자 하나씩 노드 생성
                    chunk = p[i]
                    i += 1
                    existing_child = next((c for c in node.children if c.name == chunk), None)
                    if existing_child:
                        node = existing_child
                    else:
                        node = node.add_child(name=chunk)
    return root

tree = lineage_to_newick(lineages)

# 3. Newick 파일로 저장
output_path = "Polytomy/Codes/nwk/c_elegans_lineage_tree_resolved.nwk"
tree.write(format=1, outfile=output_path)

print(f"트리 생성 완료. 저장 위치: {output_path}")

## 1.Read Raw data -> `.nwk` Tree

In [4]:
import os
os.getcwd()
os.chdir('/Users/yongjunchoi/Documents/GitHub/Research/Research')

In [5]:
import pandas as pd
from ete3 import Tree

# 0. 파일 경로
file_path = "Polytomy/Codes/Data/GSE126954_cell_annotation.csv.gz"

# 1. 데이터 읽기
df = pd.read_csv(file_path, compression="gzip")
lineages = df['lineage'].dropna().unique()  # 결측 제외, 고유 lineage

# 2. lineage -> Newick tree 변환 (대문자 덩어리, 소문자 개별 노드)
def lineage_to_newick(lineages):
    root = Tree(name="zygote")
    
    for lin in lineages:
        # '/'가 있으면 여러 경로로 분리
        paths = lin.split('/')
        for p in paths:
            node = root
            i = 0
            while i < len(p):
                # 대문자인 경우: 연속 대문자 덩어리 추출
                if p[i].isupper():
                    j = i + 1
                    while j < len(p) and p[j].isupper():
                        j += 1
                    chunk = p[i:j]  # 대문자 덩어리
                    i = j
                    # 자식 존재 여부 확인 후 생성
                    existing_child = next((c for c in node.children if c.name == chunk), None)
                    if existing_child:
                        node = existing_child
                    else:
                        node = node.add_child(name=chunk)
                else:
                    # 소문자는 글자 하나씩 노드 생성
                    chunk = p[i]
                    i += 1
                    existing_child = next((c for c in node.children if c.name == chunk), None)
                    if existing_child:
                        node = existing_child
                    else:
                        node = node.add_child(name=chunk)
    return root

tree = lineage_to_newick(lineages)

# 3. Newick 파일로 저장
output_path = "Polytomy/Codes/nwk/c_elegans_lineage_tree.nwk"
tree.write(format=1, outfile=output_path)

print(f"트리 생성 완료. 저장 위치: {output_path}")

트리 생성 완료. 저장 위치: Polytomy/Codes/nwk/c_elegans_lineage_tree.nwk


In [6]:
from ete3 import Tree, TreeStyle

# 1. 저장된 Newick 파일 불러오기
tree = Tree("Polytomy/Codes/nwk/c_elegans_lineage_tree.nwk", format=1)

# 2. TreeStyle 설정
ts = TreeStyle()
ts.show_leaf_name = True       # 리프 노드 이름 표시
ts.show_branch_length = False  # branch length 표시 여부
ts.show_branch_support = False # branch support 표시 여부
ts.mode = "r"                  # 'c' = circular, 'r' = rectangular (default)
ts.scale = 20                  # 노드 간 간격 조정

# 3. 트리 시각화
tree.show(tree_style=ts)

## 1.2. `.nwk` Tree -> Polytomy

1. Among leaves, select `n` leaves for polytomy. 
2. Polytomy leaves의 MRCA 확인.
3. Polytomy leaves를 일단 제거.
4. Polytomy leaves MRCA에 polytomy 추가

### 전처리: Polytomy 제거

In [18]:
from ete3 import Tree

# 트리 불러오기
tree = Tree("Polytomy/Codes/nwk/c_elegans_lineage_tree.nwk", format=1)

# root 바로 아래 자식 노드 이름 출력
print("Root 아래 자식 노드들:")
for child in tree.children:
    print(f"- {child.name if child.name else '(internal node)'}")

Root 아래 자식 노드들:
- MS
- D
- AB
- E
- C
- Z
- 2


In [7]:
from ete3 import Tree

# ----------------------------
# 1. Newick 파일 불러오기
# ----------------------------
tree = Tree("Polytomy/Codes/nwk/c_elegans_lineage_tree.nwk", format=1)

# ----------------------------
# 2. Polytomy 제거 함수
# ----------------------------
def remove_polytomy(node):
    # Post-order traversal
    for child in list(node.children):
        remove_polytomy(child)

    # Polytomy 제거 (child >= 3)
    if len(node.children) >= 3:
        for child in list(node.children):
            node.remove_child(child)

# ----------------------------
# 3. 트리 처리 실행
# ----------------------------
remove_polytomy(tree)

# ----------------------------
# 4. 최종 Newick 출력 및 파일 저장
# ----------------------------
output_file = "Polytomy/Codes/nwk/c_elegans_lineage_tree_polytomy_removed.nwk"
tree.write(outfile=output_file)
print(f"Polytomy-removed tree saved to {output_file}")

Polytomy-removed tree saved to Polytomy/Codes/nwk/c_elegans_lineage_tree_polytomy_removed.nwk


# Worm atlas

In [ ]:
import pandas as pd

cell_data = pd.read_csv("Polytomy/Codes/Data/Complete_cell_list.csv")

lineages = cell_data[''].tolist()